In [ ]:
from sentence_transformers import SentenceTransformer, util

# 1. Загружаем мультиязычную модель
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 2. Определяем тексты для сравнения (можно на разных языках)
text1 = "Функция, которая позволяет очищать духовой шкаф без каких-либо действий со стороны пользователя. Закройте дверцу духовки и установите программу пиролитической очистки. Духовой шкаф заблокируется и начнет набирать температуру 480-500°С. При такой температуре весь жир, который отложился на стенках духового шкафа, сгорает и превращается в пепел. Программа длится около 3 часов. Как только температура в рабочей камере опустится ниже 180°С, дверца разблокируется. Вам остается только смахнуть салфеткой остатки пепла. Результат - абсолютно чистая эмаль без лишний усилий."
text2 = "Духовой шкаф Graude BMP 60.4 S оснащен пиролитической очисткой, при которой прибор заблокируется и начнет набирать температуру 480-500 °С. Весь жир, который отложился на стенках духового шкафа, сгорает и превращается в пепел. Вам остается только смахнуть салфеткой остатки пепла. Результат – абсолютно чистая эмаль без лишних усилий. Автоматические программы приготовления Pro Cook помогут с рецептами. Управление осуществляется с помощью TFT-дисплея высокого разрешения диагональю 7”. Противень AirFry гарантирует наибольший доступ циркулируемого горячего воздуха к еде для создания хрустящей корочки. Блюда, приготовленные в AirFry гораздо менее вредные, чем приготовленные в классической фритюрнице."

# 3. Преобразуем тексты в векторы (эмбеддинги)
embedding1 = model.encode(text1)
embedding2 = model.encode(text2)

# 4. Вычисляем косинусное сходство
similarity = util.cos_sim(embedding1, embedding2)

print(f"Степень сходства: {similarity.item():.4f}")


Степень сходства: 0.7895


In [6]:
from sentence_transformers import SentenceTransformer, util

def find_unmatched_lines(file1_path: str, file2_path: str, threshold: float = 0.6):
    """
    Читает непустые строки из двух файлов и ищет для каждой строки из первого файла
    строку из второго со степенью сходства > threshold.
    Возвращает список строк из первого файла, для которых аналога не нашлось.
    """

    # Загружаем модель (однократно)
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

    # Читаем непустые строки
    def read_nonempty(path):
        with open(path, 'r', encoding='utf-8') as f:
            return [line.strip() for line in f if line.strip()]

    lines1 = read_nonempty(file1_path)
    lines2 = read_nonempty(file2_path)

    if not lines1:
        return []
    if not lines2:
        return lines1[:]  # если второй файл пуст — все строки unmatched

    # Эмбеддинги
    emb1 = model.encode(lines1, convert_to_tensor=True)
    emb2 = model.encode(lines2, convert_to_tensor=True)

    # Матрица косинусных сходств: shape [len(lines1), len(lines2)]
    sim_matrix = util.cos_sim(emb1, emb2)

    unmatched = []
    for i, line in enumerate(lines1):
        # Находим максимум по строке
        max_sim = float(sim_matrix[i].max())

        if max_sim <= threshold:
            unmatched.append(line)

    return unmatched

path_text = "Сео-слова.txt"
path_desc = "Размеченный текст.txt"
unmatched = find_unmatched_lines(path_text, path_desc, threshold=0.5)
for line in unmatched:
    print(line)